In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

GENDER RECOGNITION BY VOICE USING PYTHON

In [ ]:
#Import les bibliothéques pour les manipulation des données, pour le calcul scientifique,visualisation et graphiques.
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

LIRE LE FICHIER DES DONNEES

In [ ]:
df=pd.read_csv("../input/voicegender/voice.csv")
df.head()


    meanfreq: mean frequency (in kHz)
    sd: standard deviation of frequency
    median: median frequency (in kHz)
    Q25: first quantile (in kHz)
    Q75: third quantile (in kHz)
    IQR: interquantile range (in kHz)
    skew: skewness (see note in specprop description)
    kurt: kurtosis (see note in specprop description)
    sp.ent: spectral entropy
    sfm: spectral flatness
    mode: mode frequency
    centroid: frequency centroid (see specprop)
    peakf: peak frequency (frequency with highest energy)
    meanfun: average of fundamental frequency measured across acoustic signal
    minfun: minimum fundamental frequency measured across acoustic signal
    maxfun: maximum fundamental frequency measured across acoustic signal
    meandom: average of dominant frequency measured across acoustic signal
    mindom: minimum of dominant frequency measured across acoustic signal
    maxdom: maximum of dominant frequency measured across acoustic signal
    dfrange: range of dominant frequency measured across acoustic signal
    modindx: modulation index. Calculated as the accumulated absolute difference between adjacent measurements of fundamental frequencies divided by the frequency range
    label: male or female


In [ ]:
df.columns

POUR AFFICHER L'ENSEMBLE DE TABLE

In [ ]:
from IPython.core.display import HTML
display(HTML(df.head(10).to_html()))

**DONNEES**

In [ ]:
#à indiquer la forme(range, colonne)
df.shape

In [ ]:
#Verifier si il ya des colonnes sont bien alignés
df.describe()

In [ ]:
#Vérifier les types des données
df.info()

In [ ]:
#Identfier combien des voix sont masculin et féminin dans la base de données avant le test
df.label.value_counts()

In [ ]:
#vérifier si ila ya certaines valeurs nulle ou pas 
df.isnull().values.any()

**VISUALISATIONS**

In [ ]:
#Identification des critérés retenir
sns.pairplot(df,hue="label")

IQR,Meanfun,Q25 ont été choisi car les données masculin et féminin sont bien distincts.

In [ ]:
#Joint plot pour visualiser dans un plan de distributions d'un couple de paramètres
sns.jointplot("meanfreq","sd",df,kind="kde");

In [ ]:
plt.figure(figsize=(8,8))
sns.kdeplot(df.meanfreq,df.sd,shade=True)

In [ ]:
homme= df.label=='male'
femme= df.label=='female'
plt.figure(figsize=(8,8))
sns.kdeplot(df[homme].meanfreq, df[homme].sd, cmap="Blues",  shade=True, alpha=0.3, shade_lowest=False)
sns.kdeplot(df[femme].meanfreq, df[femme].sd, cmap="Reds", shade=True, alpha=0.3, shade_lowest=False)

In [ ]:
sns.boxplot(x="label", y="sd", data=df)

In [ ]:
sns.violinplot(x="label",y="meanfreq",data=df,scale="width")

In [ ]:
fig = sns.FacetGrid(df, hue="label", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "meanfreq", shade=True)
fig.add_legend()

In [ ]:
sns.lmplot(x="meanfreq", y="sd", data=df, fit_reg=False, hue='label')

MACHINE LEARNING


On sépare le dataset en deux parties :

    un ensemble d'apprentissage (entre 70% et 90% des données), qui va permettre d'entraîner le modèle
    un ensemble de test (entre 10% et 30% des données), qui va permettre d'estimer la pertinence de la prédiction

In [ ]:
data_train = df.sample(frac=0.8, random_state=1)          # 80% des données avec frac=0.8
data_test =df.drop(data_train.index)     # le reste des données pour le test

On sépare les données d'apprentissage (X_train) et la cible (y_train, la colonnes des données classe)

In [ ]:
x_train = data_train.drop(['label'], axis=1)
y_train = data_train['label']
x_test = data_test.drop(['label'], axis=1)
y_test = data_test['label']


**Régression logistique**

On veut prédire une variable aléatoire Y
à partir d'un vecteur de variables explicatives X=(X1,...,Xn)

On

La fonction logistique ex1+ex
varie entre −∞ et +∞ pour x variant entre 0 et 1.
Elle est souvent utilisée pour "mapper" une probabilité et un espace réel

In [ ]:
plt.figure(figsize=(9,9))

logistique = lambda x: np.exp(x)/(1+np.exp(x))   

x_range = np.linspace(-10,10,50)       
y_values = logistique(x_range)

plt.plot(x_range, y_values, color="red")

La régression logistique consiste à trouver une fonction linéaire C(X) qui permette d'estimer la probabilité de Y=1 connaissant X :
p(Y=1|X)=eC(X)1+eC(X)

Autrement dit, cela revient à trouver une séparation linéaire des caractéristiques qui minimise un critère d'erreur.

Pour plus de détails, cf par exemple :
http://eric.univ-lyon2.fr/~ricco/cours/cours/pratique_regression_logistique.pdf

On peut tracer la courbe de régression logistique pour prédire l'espèce Virginica à partir de la longueur du sépale avec la fonction lmplot :

On veut maintenant prédire l'espèce à partir de toutes les caractéristiques, et évaluer la qualité de cette prédiction en utilisant la régression logistique définie dans la librairie sklearn :


In [ ]:
from sklearn.linear_model import LogisticRegression

On entraîne le modèle de régression logistique avec fit 

In [ ]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(max_iter=500)
log.fit(x_train,y_train)

On peut prédire les valeurs sur l'ensemble de test avec le modèle entraîné :

In [ ]:
y_lr = log.predict(x_test)

**Score et matrice de confusion**

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

La mesure de pertinence compte le nombre de fois où l'algorithme a fait une bonne prédiction (en pourcentage) :

In [ ]:
lr_score = accuracy_score(y_test, y_lr)
print(lr_score)

Une mesure plus fine consiste à compter le nombre de faux positif (valeur prédite 1 et réelle 0) et de vrai négatif (valeur prédite 0 et réelle 1). On utilise une matrice de confusion :

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_test, y_lr)
print(cm)

In [ ]:
pd.crosstab(y_test, y_lr, rownames=['Reel'], colnames=['Prediction'], margins=True)

Le résultat est il-satisfaisant?
Quel pourrait etre le problème?

92,5% est le résultat du log regression. Mais dans le fichier données , l'étude sur les données en utilisant Log régression a eu un taux de précision de 71%. Donc le résultat est satisfaisant.
Recherche comparaison:http://www.primaryobjects.com/2016/06/22/identifying-the-gender-of-a-voice-using-machine-learning/

**ARBRES DE DECISION**

In [ ]:
fig = sns.FacetGrid(df, hue="label", aspect=3) # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "meanfun", shade=True)
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(df[df.meanfun>0.14], hue="label", aspect=3) # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "Q25", shade=True)
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(df[(df.meanfun>0.14)&(df.Q25>0.17)], hue="label", aspect=3) # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "IQR", shade=True)
fig.add_legend()

L'indice GINI mesure avec quelle fréquence un élément aléatoire de l'ensemble serait mal classé si son étiquette était sélectionnée aléatoirement depuis la distribution des étiquettes dans le sous-ensemble.

In [ ]:
from sklearn import tree
dtc = tree.DecisionTreeClassifier()
dtc.fit(x_train,y_train)
y_dtc = dtc.predict(x_test)
print(accuracy_score(y_test, y_dtc))

In [ ]:
plt.figure(figsize=(30,30))
tree.plot_tree(dtc, feature_names=x_train.columns, class_names=['homme','femme'], fontsize=14, filled=True)  

On peut modifier certains paramètres : Le paramètre max_depth est un seuil sur la profondeur maximale de l’arbre. Le paramètre min_samples_leaf donne le nombre minimal d’échantillons dans un noeud feuille.

In [ ]:
dtc1 = tree.DecisionTreeClassifier(max_depth = 3, min_samples_leaf = 20)
dtc1.fit(x_train,y_train)

On obtient un arbre un peu différent :

In [ ]:
plt.figure(figsize=(10,10))
tree.plot_tree(dtc1, feature_names=x_train.columns, class_names=['homme','femme'], fontsize=14, filled=True)  

In [ ]:
y_dtc1 = dtc1.predict(x_test)
print(accuracy_score(y_test, y_dtc1))

RANDOM FORESTS

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(x_train, y_train)
y_rf = rf.predict(x_test)

In [ ]:
rf_score = accuracy_score(y_test, y_rf)
print(rf_score)

In [ ]:
#Et la matrice de confusion 
pd.crosstab(y_test, y_rf, rownames=['Reel'], colnames=['Prediction'], margins=True)



L'attribut featureimportances renvoie un tableau du poids de chaque caractéristique dans la décision :


In [ ]:
importances = rf.feature_importances_
indices = np.argsort(importances)
plt.figure(figsize=(12,8))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), df.columns[indices])
plt.title('Importance des caracteristiques')